# Introduction to TileMatrixSets

The goal of this notebook is to give a quick introduction to the new TMS option in rio-tiler.

# Requirements

To be able to run this notebook you'll need the following requirements:
- rio-tiler~= 3.0
- ipyleaflet

In [ ]:
# !pip install rio-tiler
# !pip install ipyleaflet

In [ ]:
import morecantile
from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    TileLayer,
    WMSLayer,
    GeoJSON,
    projections
)

## Data

For this demo we will use some NAIP data hosted on Azure.

https://azure.microsoft.com/fr-fr/services/open-datasets/catalog/naip/


The data is similar to the data hosted on [AWS](https://registry.opendata.aws/naip/), but using the one on Azure is easier because it offers direct `http` access without needing an AWS account.

In [ ]:
# For this DEMO we will use this file
src_path = "https://naipblobs.blob.core.windows.net/naip/v002/al/2019/al_60cm_2019/30087/m_3008701_ne_16_060_20191115.tif"

## Tile Server

For this demo, we need to create a minimal tile server.

In [ ]:
import json
from concurrent import futures

from tornado import web
from tornado import gen
from tornado.httpserver import HTTPServer
from tornado.concurrent import run_on_executor

from rio_tiler.io import COGReader
from rio_tiler.errors import TileOutsideBounds
from rio_tiler.profiles import img_profiles

class TileServer:
    
    def __init__(self, src_path):
        """Initialize Tornado app."""
        self.server = None
        self.app = web.Application([
            (r"^/tiles/(\w+)/(\d+)/(\d+)/(\d+)", TileHandler, {"url": src_path}),
        ])

    def start(self):
        """Start tile server."""
        self.server = HTTPServer(self.app)
        self.server.listen(8080)
  
    def stop(self):
        """Stop tile server."""
        if self.server:
            self.server.stop()


class TileHandler(web.RequestHandler):
    """Tile requests handler."""

    executor = futures.ThreadPoolExecutor(max_workers=16)

    def initialize(self, url):
        """Initialize tiles handler."""
        self.url = url

    @run_on_executor
    def _get_tile(self, tms, z, x, y):

        try:
            with COGReader(self.url, tms=morecantile.tms.get(tms)) as cog:
                img = cog.tile(x, y, z, indexes=(1,2,3))
        except TileOutsideBounds:
            raise web.HTTPError(404)

        prof = img_profiles.get("PNG", {})
        return img.render(img_format="PNG", **prof)

    @gen.coroutine
    def get(self, tms, z, x, y):
        """Retunrs tile data and header."""
        self.set_header("Access-Control-Allow-Origin", "*")
        self.set_header("Access-Control-Allow-Methods", "GET")
        self.set_header("Content-Type", "image/png")
        self.set_header("Cache-Control", "no-store, no-cache, must-revalidate")
        res = yield self._get_tile(tms, int(z), int(x), int(y))
        self.write(res)


ts = TileServer(src_path)
ts.start()

List the supported TMS from morecantile

In [ ]:
print("Supported TMS:")
for name, tms in morecantile.tms.tms.items():
    print("-", name, "|",tms.crs)

## WebMercator TMS

In [ ]:
with COGReader(src_path) as cog:
    info = cog.info()
print(info.dict(exclude_none=True))

m = Map(center=(30.96, -87.90), zoom=info.minzoom, basemap={})

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/WebMercatorQuad/{z}/{x}/{y}",
    min_zoom=info.minzoom,
    max_zoom=info.maxzoom,
    opacity=1,
)
m.add_layer(layer)
m

## WGS84 TMS

In [ ]:
with COGReader(src_path, tms=morecantile.tms.get("WorldCRS84Quad")) as cog:
    info = cog.info()
print(info.dict(exclude_none=True))

m = Map(center=(30.96, -87.90), zoom=info.minzoom, basemap={}, crs=projections.EPSG4326)

layer = TileLayer(
    url="http://127.0.0.1:8080/tiles/WorldCRS84Quad/{z}/{x}/{y}",
    min_zoom=info.minzoom,
    max_zoom=info.maxzoom,
    opacity=1,
)
m.add_layer(layer)
m

In [ ]:
ts.stop()